In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install pypdf
!pip install chromadb
!pip install langchainhub

In [ ]:
import os

# Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

# from dotenv import load_dotenv
# make sure you have python-dotenv installed
# load_dotenv("./.env")

Let's set up a study workflow using Jupyter Notebooks, LLMs, and langchain.

In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [2]:
pdf_path = "./assets-resources/attention-paper.pdf"

loader = PyPDFLoader(pdf_path) # LOAD
pdf_docs = loader.load_and_split() # SPLIT
embeddings = OpenAIEmbeddings() # EMBED
vectordb = Chroma.from_documents(pdf_docs, embedding=embeddings) # STORE
retriever = vectordb.as_retriever()
llm = ChatOpenAI(model="gpt-4-1106-preview")
pdf_qa = RetrievalQA.from_llm(llm=llm, retriever=retriever) # RETRIEVE
pdf_qa

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12d1be110>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12d83fe50>, model_name='gpt-4-1106-preview', openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy='')), document_prompt=PromptTemplate(input_variables=['page_content'], template='Context:\n{page_content}'), document_variable_name='context'), re

In [3]:
query = "What are the key components of the transformer architecture?"
result = pdf_qa.invoke({"query": query, "chat_history": []})

In [4]:
print(result["result"])

The key components of the Transformer architecture are as follows:

1. **Encoder and Decoder Stacks**: The Transformer model consists of an encoder and a decoder, each made up of N=6 identical layers. 

2. **Self-Attention Mechanism**: This allows the model to weigh the influence of different parts of the input data differently and is a key component in both the encoder and decoder layers.

3. **Multi-Head Attention**: To capture different aspects of information from the input sequence, the Transformer uses multiple attention heads, allowing the model to focus on different positions of the input sequence simultaneously.

4. **Position-Wise Feed-Forward Networks**: Each layer in the encoder and decoder contains a fully connected feed-forward network that is applied to each position separately and identically.

5. **Positional Encoding**: Since the model lacks recurrence and convolution, it uses positional encodings to give the model some information about the order of the sequence. This

In [5]:
def ask_pdf(pdf_qa,query):
    print("QUERY: ",query)
    result = pdf_qa.invoke({"query": query, "chat_history": []})
    answer = result["result"]
    print("ANSWER", answer)
    return answer


ask_pdf(pdf_qa,"How does the self-attention mechanism in transformers differ from traditional sequence alignment methods?")

QUERY:  How does the self-attention mechanism in transformers differ from traditional sequence alignment methods?
ANSWER The self-attention mechanism in transformers differs from traditional sequence alignment methods in several key ways:

1. **Global Dependencies**: Self-attention allows the model to consider the entire sequence simultaneously, enabling each position to attend to all positions in the previous layers of the network. This contrasts with traditional sequence alignment methods in RNNs, where the hidden state at each position is computed based on the previous hidden state and the current input, making it harder to capture long-range dependencies.

2. **Parallelization**: The self-attention mechanism enables parallel processing of the sequence data, unlike RNNs which process data sequentially. This parallelization significantly speeds up training because it eliminates the need to wait for the computation of the previous state before processing the current state.

3. **Fixed

"The self-attention mechanism in transformers differs from traditional sequence alignment methods in several key ways:\n\n1. **Global Dependencies**: Self-attention allows the model to consider the entire sequence simultaneously, enabling each position to attend to all positions in the previous layers of the network. This contrasts with traditional sequence alignment methods in RNNs, where the hidden state at each position is computed based on the previous hidden state and the current input, making it harder to capture long-range dependencies.\n\n2. **Parallelization**: The self-attention mechanism enables parallel processing of the sequence data, unlike RNNs which process data sequentially. This parallelization significantly speeds up training because it eliminates the need to wait for the computation of the previous state before processing the current state.\n\n3. **Fixed Number of Operations**: The number of operations required to relate signals from two arbitrary input or output po

In [6]:
quiz_questions = ask_pdf(pdf_qa, "Quiz me with 3 simple questions on the positional encodings and the role they play in transformers.")

quiz_questions

QUERY:  Quiz me with 3 simple questions on the positional encodings and the role they play in transformers.
ANSWER Sure, here are three simple questions about positional encodings in the context of the Transformer model:

1. What is the purpose of positional encodings in the Transformer model architecture?
   - A) To add information about the absolute or relative position of the tokens in the sequence.
   - B) To replace the attention mechanism.
   - C) To increase the model size.

2. How are positional encodings combined with the input embeddings in the Transformer model before processing by the encoder and decoder stacks?
   - A) By concatenating the positional encoding vector to the input embedding vector.
   - B) By adding the positional encoding vector to the input embedding vector.
   - C) By multiplying the input embedding vector with the positional encoding vector.

3. What alternative to the original sinusoidal positional encodings was mentioned in the context provided, and wh

'Sure, here are three simple questions about positional encodings in the context of the Transformer model:\n\n1. What is the purpose of positional encodings in the Transformer model architecture?\n   - A) To add information about the absolute or relative position of the tokens in the sequence.\n   - B) To replace the attention mechanism.\n   - C) To increase the model size.\n\n2. How are positional encodings combined with the input embeddings in the Transformer model before processing by the encoder and decoder stacks?\n   - A) By concatenating the positional encoding vector to the input embedding vector.\n   - B) By adding the positional encoding vector to the input embedding vector.\n   - C) By multiplying the input embedding vector with the positional encoding vector.\n\n3. What alternative to the original sinusoidal positional encodings was mentioned in the context provided, and what was the observed effect on model performance when using this alternative?\n   - A) Learned position

In [7]:
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.0)

In [8]:
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = f"You take in text and spit out Python code doing what the user wants"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template("Return ONLY a PYTHON list containing the questions in this text: {questions}")

In [9]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

In [10]:
quiz_chain = chat_prompt | llm

In [11]:
quiz_chain.invoke({"questions": quiz_questions})

AIMessage(content='```python\nquestions = [\n    "What is the purpose of positional encodings in the Transformer model architecture?",\n    "How are positional encodings combined with the input embeddings in the Transformer model before processing by the encoder and decoder stacks?",\n    "What alternative to the original sinusoidal positional encodings was mentioned in the context provided, and what was the observed effect on model performance when using this alternative?"\n]\n```', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 266, 'total_tokens': 348}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7dab1b6f-fe80-4946-8489-246019018e66-0', usage_metadata={'input_tokens': 266, 'output_tokens': 82, 'total_tokens': 348})

In [12]:
import re

def extract_python_code(markdown_string):
    pattern = r'```python\n(.*?)\n```'
    matches = re.findall(pattern, markdown_string, re.DOTALL)

    if matches:
        python_code = matches[0]
        return python_code
    else:
        return None

In [13]:
from langchain_core.runnables import RunnableLambda


quiz_chain = chat_prompt | llm | RunnableLambda(lambda x: x.content) | extract_python_code

Disclaimer: We haven't discussed runnable at length, but essentially they make up the core of the LCEL interface. 

`RunnableLambda` allows you to take in an output from part of the chain and pass it along after performing some transformation defined withint its lambda function.

In [14]:
questions_list = quiz_chain.invoke({"questions": quiz_questions})

In [15]:
questions_list

'questions = [\n    "What is the purpose of positional encodings in the Transformer model architecture?",\n    "How are positional encodings combined with the input embeddings in the Transformer model before processing by the encoder and decoder stacks?",\n    "What alternative to the original sinusoidal positional encodings was mentioned in the context provided, and what was the observed effect on model performance when using this alternative?"\n]'

In [16]:
exec(questions_list)

In [17]:
questions

['What is the purpose of positional encodings in the Transformer model architecture?',
 'How are positional encodings combined with the input embeddings in the Transformer model before processing by the encoder and decoder stacks?',
 'What alternative to the original sinusoidal positional encodings was mentioned in the context provided, and what was the observed effect on model performance when using this alternative?']

In [18]:
# the questions variable was created within the string inside the `questions_list` variable.
answers = []
for q in questions:
    answers.append(ask_pdf(pdf_qa,q))

QUERY:  What is the purpose of positional encodings in the Transformer model architecture?
ANSWER Positional encodings in the Transformer model architecture serve the purpose of giving the model information about the order of the words in the sequence. Since the Transformer relies solely on attention mechanisms and does not use any recurrence (like RNNs) or convolution, it does not inherently consider the position of words in the sequence. Positional encodings are added to the input embeddings to provide the model with some notion of word order, which is crucial for many language-related tasks where the order of words affects the meaning.

The positional encodings can take various forms, but in the original "Attention Is All You Need" paper by Vaswani et al., they used sine and cosine functions of different frequencies to generate these positional encodings. These are designed so that each position would have a unique encoding and the relative positions could be easily interpreted by t

In [19]:
evaluations = []

for q,a in zip(questions, answers):
    # Check for results
    evaluations.append(ask_pdf(pdf_qa,f"Is this: {a} the correct answer to this question: {q} according to the paper? Return ONLY '''YES''' or '''NO'''. Output:"))

evaluations

QUERY:  Is this: Positional encodings in the Transformer model architecture serve the purpose of giving the model information about the order of the words in the sequence. Since the Transformer relies solely on attention mechanisms and does not use any recurrence (like RNNs) or convolution, it does not inherently consider the position of words in the sequence. Positional encodings are added to the input embeddings to provide the model with some notion of word order, which is crucial for many language-related tasks where the order of words affects the meaning.

The positional encodings can take various forms, but in the original "Attention Is All You Need" paper by Vaswani et al., they used sine and cosine functions of different frequencies to generate these positional encodings. These are designed so that each position would have a unique encoding and the relative positions could be easily interpreted by the model. The Transformer model can, therefore, learn to use these signals to und

['YES', 'YES', 'YES']

In [20]:
scores = []

yes_count = evaluations.count('YES')
scores.append(yes_count)
print(scores)

[3]
